## EXTRAER DATOS DEL ETABS

## Librerías

In [1]:
import comtypes.client
import pandas as pd
import numpy as np
import sys
import matplotlib.pyplot as plt

## Función para conectarnos al ETABS
Esta función se puede usar para cualquier programa de ETABS que se tenga abierto y ejecutado

In [71]:
def connect_to_etabs():
    helper = comtypes.client.CreateObject('ETABSv1.Helper')
    helper = helper.QueryInterface(comtypes.gen.ETABSv1.cHelper)
    try:
        myETABSObject = helper.GetObject("CSI.ETABS.API.ETABSObject")
        print("Connected to ETABS model")
    except (OSError, comtypes.COMError):
        print("No running instance of the program found or failed to attach.")
        sys.exit(-1)
    SapModel = myETABSObject.SapModel
    return SapModel,myETABSObject,helper

SapModel, myETABSObject, helper = connect_to_etabs()

Connected to ETABS model


## Extracción de Masa Participativa

In [53]:
def MasasParticipativas(SapModel):
    name_table = "Modal Participating Mass Ratios"
    table = SapModel.DatabaseTables.GetTableForDisplayArray(name_table,GroupName="")
    cols = table[2]
    noOfrows = table[3]
    vals = np.array_split(table[4],noOfrows)
    df = pd.DataFrame(vals)
    df.columns = cols
    #Eliminar Columnas del Dataframe df
    df.drop(["UZ","RX","RY","SumRX","SumRY","SumUZ","Case"],axis=1, inplace=True)
    #Mover la columna "RZ" a la columna que ocupa la posición 4
    df.insert(4,'RZ',df.pop('RZ'))
    display(df)
    # Dirección X-X
    posicion_x = df["UX"].idxmax()
    modox = df.loc[posicion_x,"Mode"]
    ux = df.loc[posicion_x,"UX"]
    Tx = df.loc[posicion_x,"Period"]
    print(f"La Masa Participativa Predominante en la Dirección X-X es {ux} con un Periodo de {Tx} seg en el Modo {modox}")

    # Dirección Y-Y
    posicion_y = df["UY"].idxmax()
    modoy = df.loc[posicion_y,"Mode"]
    uy = df.loc[posicion_y,"UY"]
    Ty = df.loc[posicion_y,"Period"]
    print(f"La Masa Participativa Predominante en la Dirección Y-Y es {uy} con un Periodo de {Ty} seg en el Modo {modoy}")
    

In [54]:
MasasParticipativas(SapModel)

,Mode,Period,UX,UY,RZ,SumUX,SumUY,SumRZ
0,1,0.419,0,0.8314,0,0,0.8314,0
1,2,0.398,0.8373,0,0,0.8373,0.8314,0
2,3,0.38,0,0,0.8368,0.8373,0.8314,0.8368
3,4,0.123,0,0.1176,0,0.8373,0.949,0.8368
4,5,0.118,0.1139,0,0,0.9513,0.949,0.8368
5,6,0.113,0,0,0.1139,0.9513,0.949,0.9506
6,7,0.061,0,0.0407,0,0.9513,0.9897,0.9506
7,8,0.06,0.0389,0,0,0.9902,0.9897,0.9506
8,9,0.057,0,0,0.0394,0.9902,0.9897,0.99
9,10,0.038,0,0.0103,0,0.9902,1,0.99


La Masa Participativa Predominante en la Dirección X-X es 0.8373 con un Periodo de 0.398 seg en el Modo 2
La Masa Participativa Predominante en la Dirección Y-Y es 0.8314 con un Periodo de 0.419 seg en el Modo 1


# Extracción de Solicitaciones en Vigas

In [103]:
def SolicitacionesVigas(SapModel, Piso="Story1", viga="B16", Data = False):
    name_table = "Element Forces - Beams"
    kN_m_C = 6
    ret = SapModel.SetPresentUnits(kN_m_C)

    table = SapModel.DatabaseTables.GetTableForDisplayArray(name_table,GroupName="")
    cols = table[2]
    noOfrows = table[3]
    vals = np.array_split(table[4],noOfrows)
    df = pd.DataFrame(vals)
    df.columns = cols

    #Eliminar Columnas del Dataframe df
    df.drop(["CaseType","StepLabel","UniqueName","Location","Element","StepNumber","ElemStation"],axis=1, inplace=True)
    # Filtrado de Nivel
    df1 = df[df.Story.isin([Piso])]
    # Filtrado por Beam
    df1 = df1[df1.Beam.isin([viga])]
    # Filtrado por OutputCase
    df1 = df1[df1.OutputCase.isin(["ENVOLVENTE"])]
    # Filtrado de Steptype
    M_max = df1[df1.StepType.isin(["Max"])]
    M_min = df1[df1.StepType.isin(["Min"])]

    # Reset de Index
    M_max = M_max.reset_index(drop=True)
    M_min = M_min.reset_index(drop=True)

    if Data == True:
        display(M_max)
        display(M_min)
    else:
        pass
    
    # Extracción de Momentos Últimos de Diseño
    M_izq = pd.DataFrame([[M_min.loc[0,"M3"]],
            [M_max.loc[0,"M3"]]])
    M_central = pd.DataFrame([[M_min.loc[6,"M3"]],
                [M_max.loc[6,"M3"]]])
    M_der = pd.DataFrame([[M_min.loc[12,"M3"]],
            [M_max.loc[12,"M3"]]])

    df = pd.concat([M_izq,M_central,M_der],axis=1)
    df.columns = ["M_izq (kN.m)","M_central (kN.m)","M_der (kN.m)"]
    display(df)


In [106]:
SolicitacionesVigas(SapModel, Piso="Story3", viga="B16", Data = False)

,M_izq (kN.m),M_central (kN.m),M_der (kN.m)
0,-159.4364,31.1884,-151.7732
1,11.9027,68.4404,12.1899
